In [1]:
import os
import pandas as pd
import json
import random
from pathlib import Path
import traceback
from utils.utils import *
from utils.data_info import *
from utils.subject_path import *
from call_gpt.call_gpt import (
    send_chat_request,
    send_chat_request_async,
    run_multi_send_chat_request,
)
import re 
import sys
from datetime import datetime
old_stdout = sys.stdout
old_stderr = sys.stderr

In [2]:
id_file_path = "../data/sample_ids_1224_eval.json"
save_dir = "../test_data/small"
current_datetime = datetime.now()
current_data = current_datetime.strftime("%Y-%m-%d")
with open(id_file_path, 'r') as f:
    id_data = json.load(f)
save_dir = os.path.join(save_dir, current_data)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
def get_true_data(dir,subject,true_id):
    file_path = os.path.join(dir,subject, true_id+'.json')
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['components_title_json']
with open("../data/teach_mode_time.json", 'r') as f:
    teach_mode_time = json.load(f)

for subject_ch, value in id_data.items():
    data_list = []
    for item in value:
        lesson_info = item["lesson_info"]
        temp_info = lesson_info.split("_")
        version = temp_info[2]
        level = temp_info[0]
        grade = temp_info[3]
        topic = temp_info[4]
        subject = temp_info[1]
        tp_path = os.path.join(subject_textbook_path[subject], level+'_'+subject+'_'+version+'_'+grade+'.json')
        tb_value = read_json(tp_path)
        textbook = get_textbook_value(topic, tb_value)
        # print("value",value)
        # components_content = get_true_data(lesson_restructure, subject_ch, item['true_data_id'])
        mode = item['mode']
        data_item = {
            "id": item['id'],
            "lesson_info": lesson_info,
            # "true_data_id": item['true_data_id'],
            "subject": subject,
            "version": version,
            "level": level,
            "grade": grade,
            "topic": topic,
            "textbook": textbook,
            "mode":mode,
            'teach_stage':teach_mode_time[mode]['teach_stage_time'],
            # "components_content": components_content,
        }
        data_list.append(data_item)
    example_type = ['0']
    tb_type = ['0', '1']
    for et in example_type:
        for tt in tb_type:
            rst_data = create_test_data(data_list,et,tt)
            save_path = os.path.join(save_dir, 'test_data_'+subject_ch+"_example_"+et+'_tb_type_'+tt+'.json')
            with open(save_path,'w', encoding='utf-8') as f:
                json.dump(rst_data, f, ensure_ascii=False, indent=4)
            print('save_path',save_path)

In [ ]:
from datetime import datetime

# 获取当前日期和时间
current_datetime = datetime.now()

# 打印当前日期和时间
print("当前日期和时间:", current_datetime)

# 只获取当前日期
current_date = current_datetime.date()
print("当前日期:", current_date)

# 只获取当前时间
current_time = current_datetime.time()
print("当前时间:", current_time)

In [6]:
# /mnt/pfs_bj/zitao_team/zengxiaoli/data/lesson_generator/test_data
# /mnt/pfs/zitao_team/zengxiaoli/data/lesson_generator/test_data
# /mnt/pfs/zitao_team/zengxiaoli/data/lesson_generator/data/test_data_information_1014.json

In [ ]:
temp = '"科学_example_0_tb_type_0" "科学_example_0_tb_type_1" "科学_example_1_tb_type_0" "科学_example_1_tb_type_1" "英语_example_0_tb_type_0" "英语_example_0_tb_type_1" "英语_example_1_tb_type_0" "英语_example_1_tb_type_1" "物理_example_0_tb_type_0" "物理_example_0_tb_type_1" "物理_example_1_tb_type_0" "物理_example_1_tb_type_1" "化学_example_0_tb_type_0" "化学_example_0_tb_type_1" "化学_example_1_tb_type_0" "化学_example_1_tb_type_1"'
temp = temp.replace(' ',',')
temp

In [ ]:
clean_data_dir = "../result/clean_data/2025-01-19/step_one/"
tb_data_dir = "../result/kp_content/2025-01-17/core"
test_type = "example_0_tb_type_0"
save_dir = "../test_data/unit"
save_dir = get_save_dir(save_dir)
for root, dirs, files in os.walk(clean_data_dir):
    if test_type in root:
        if 'process' in root: continue
        if 'sta' in root : continue
        if 'eval' in root : continue
        data = get_data_by_unit(root,tb_data_dir)
        save_path = os.path.join(save_dir, 'unit_'+root.split('/')[-1]+'_2'+'.json')
        # print(save_path)
        print(len(data))
        rst_data =  create_unit_test_data(data,'1','1')
        # rst_data,save_path = create_single_test_data(rst_data, 'module','教学评价', save_path)
        with open(save_path, 'w') as f:
            json.dump(rst_data, f, ensure_ascii=False, indent=4)
        print("save_path",save_path)
                  

In [ ]:
data[0]

#### 将另外的prompt保存下来

In [4]:
import os
import json

root_dir = '/mnt/cfs/zengxiaoli/lesson_plan_generator_test/apimodel/GPT4-FAST_1221'
subjects = ["化学", "物理", "数学", "科学", "英语", "信息科技"]
# modules = ["module1", "module2", "module3"]  # 假设模块名称为 module1, module2, module3，请根据实际情况调整

subject_all = {}
for subject in subjects:
    subject_all[subject] = {}
    for module in modules:
        subject_all[subject][module] = {}

for root, dirs, files in os.walk(root_dir):
    subject_name = root.split('/')[-1]
    if subject_name not in subjects:
        continue
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r') as f:
                data = json.load(f)
            module_name = file.split('.')[0].split('_')[-1]
            data['model_name'] = 'Qwen72'
            if module_name in modules:  # 确保模块名称在预定义的模块列表中
                subject_all[subject_name][module_name][data['id']] = data  # 使用字典键名'id'
# print(subject_all)
save_path = "../test_data/other_module"
os.makedirs(save_path, exist_ok=True)

for subject, value in subject_all.items():
    id_module = {}
    for module in modules:
        id_module[module] = {}  # 修正这里的键名
        id_list = []
        for item_id, item in value[module].items():  # 遍历字典项
            id_module[module][item_id] = item
            id_list.append(item_id)
    
    rst = []
    for item_id in id_list:
        for module in modules:
            if item_id in id_module[module]:  # 检查是否存在该ID
                rst.append(id_module[module][item_id])
    print(len(rst))
    with open(os.path.join(save_path, subject + '.json'), 'w') as f:  # 修正文件名
        json.dump(rst, f, ensure_ascii=False, indent=4)


700
700
700
700
700
700


In [22]:
import os
import pandas as pd
import json
import random
import copy
from pathlib import Path
import traceback
from utils.utils import *
from utils.data_info import *
from utils.subject_path import *
from call_gpt.call_gpt import (
    send_chat_request,
    send_chat_request_async,
    run_multi_send_chat_request,
)
import re 
import sys
from datetime import datetime
from gen_information import *
old_stdout = sys.stdout
old_stderr = sys.stderr
save_dir = "../result/evel_data"
save_dir = get_save_dir(save_dir)
csv_path = "../data/eval_lesson_plan(1).csv"
csv_data = pd.read_csv(csv_path)
subject_dict = {}
subjects = ["化学", "物理", "数学", "科学", "英语", "信息科技"]
with open("../data/teach_mode_time.json", 'r') as f:
    teach_mode_time = json.load(f)
for index, row in csv_data.iterrows():
    subject = row['subject']
    if subject not in subjects: continue
    if subject not in subject_dict:
        subject_dict[subject] = []
    temp = row.to_dict()
    temp['lesson_info'] = '_'.join([temp['level'], temp['subject'], temp['version'], temp['grade'], temp['topic']])
    temp['teach_stage'] = teach_mode_time[temp['mode']]['teach_stage']
    subject_dict[subject].append(temp)

for key,value in subject_dict.items():
    save_path = os.path.join(save_dir, 'test_data_'+key+"_example_"+'0'+'_tb_type_'+'0'+'.json')
    with open(save_path,'w', encoding='utf-8') as f:
        json.dump(value, f, ensure_ascii=False, indent=4)

# subject_dict['科学']